In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd
import re

In [2]:
# time
t1 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t1))


'2019-05-19 17:40:03'

In [3]:
bidInfo = {}

In [4]:
csvFile = open("namelist.csv","w+")
writer = csv.writer(csvFile)

bidInfo = {}

header = ("name","date","title","consultPrice","currentPrice","bidCount","delayCount","applyCount","area","areaB","url")

writer.writerow(header)

#writer.writerow(('name','url'))



89

In [5]:
sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
                 "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  



In [6]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [7]:
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))


<selenium.webdriver.remote.webelement.WebElement (session="9e7c118c5f91d84a22bacb180625a8c2", element="0.5875080451282944-1")>

In [8]:
#if test, set  pages = 1 
#pages = 1
pages = driver.find_element_by_class_name("page-total").text

In [9]:
for page in range(1,int(pages)+1):
    if(page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    items = jsonData["data"]
    
    itemLen = len(items)
    #itemLen = 6   #test
    for item in range(itemLen):    
        bidInfo["status"] = items[item]["status"]

        
        if(bidInfo["status"] == "done"):
            
            bidInfo["id"] = items[item]["id"]
            bidInfo["title"] = items[item]["title"]
            bidInfo["consultPrice"] = items[item]["consultPrice"]
            bidInfo["currentPrice"] = items[item]["currentPrice"]
            bidInfo["bidCount"] = items[item]["bidCount"]
            bidInfo["delayCount"] = items[item]["delayCount"]
            bidInfo["applyCount"] = items[item]["applyCount"]

            url = "#pai-item-"+str(bidInfo["id"])
            # Click the link which opens in a new window
            driver.find_element_by_css_selector(url).click()
            time.sleep(1) # have to wait sometime

            # Wait for the new window or tab
            wait.until(EC.number_of_windows_to_be(2))
            handles = driver.window_handles
            driver.switch_to.window(handles[1])
            # size
            
            urlNew = driver.current_url
            url2 = str(urlNew)+"#J_Confirmation"
            driver.get(url2)
            
            
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))
            info = driver.find_element_by_class_name("c-content").text
            index = info.find("通过")
            
            # getName
            bidInfo["name"] = info[4:index]
            # getDate
            bidInfo["date"] = driver.find_element_by_class_name("c-stamp").text
            

            #print
            print("Name: "+bidInfo["name"]+": Page :"+str(page)+"   Item :"+str(item))



           # get area
            driver.get(str(urlNew)+"#NoticeDetail")
            time.sleep(1)
            notice = driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent')            
            left = notice.find("建筑")
            right = notice.find("方米",left)
            
            if(left != -1 and right != -1 ):
                size = notice[(left+2) : right]
                areas = re.findall(r"\d+\.\d*",size)
                if(len(areas) > 0):
                    bidInfo["area"]  = areas[0]
                else:
                    bidInfo["area"]  = 0
            else:
                bidInfo["area"]  = 0

            
            bidInfo["url"] = urlNew

           ###  Tip
            driver.get(str(urlNew)+"#RemindTip")
            time.sleep(1)
            tip  = driver.find_element_by_id("J_desc").get_attribute('textContent')
            tip_left = tip.find("建筑")
            tip_right =  tip.find("方米",tip_left)
            
            if(tip_left != -1 and tip_right != -1 ):
                size = tip[(tip_left+2) : tip_right]           
                areaB = re.findall(r"\d+\.\d*",size)
                if(len(areaB) > 0 ):
                    bidInfo["areaB"] =areaB[0]
                else:
                    bidInfo["areaB"] = 0
            else:
                bidInfo["areaB"] = 0

### write into file
            writer.writerow((bidInfo["name"],bidInfo["date"],bidInfo["title"],
                                   bidInfo["consultPrice"],bidInfo["currentPrice"],
                                   bidInfo["bidCount"],bidInfo["delayCount"],bidInfo["applyCount"],
                                   bidInfo["area"],bidInfo["areaB"],bidInfo["url"]
                                 ))
                                 

            driver.close()
            driver.switch_to.window(handles[0])
        else:
            pass
           # print("***item failed******"+": Page :"+str(page)+"   Item :"+str(item))



Name: 杨宁: Page :1   Item :4
Name: 魏环利: Page :1   Item :5
Name: 李玉芹: Page :1   Item :6
Name: 于德龙: Page :1   Item :7
Name: 张丹琪: Page :1   Item :9
Name: 郭慧杰: Page :1   Item :28
Name: 夏建明: Page :1   Item :29
Name: 陈健: Page :1   Item :30
Name: 朱秀娥: Page :2   Item :0
Name: 只悦强: Page :2   Item :2
Name: 刘倩南: Page :2   Item :3
Name: 张晨: Page :2   Item :4
Name: 徐丽雅: Page :2   Item :5
Name: 张毅: Page :2   Item :6
Name: 王晨: Page :2   Item :10
Name: 倪秀华: Page :2   Item :12
Name: 朱艳秋: Page :2   Item :16
Name: 郑学明: Page :2   Item :18
Name: 赵轶鸣: Page :2   Item :21
Name: 侯建林: Page :2   Item :22
Name: 辛国帅: Page :3   Item :5
Name: 周元松: Page :3   Item :6
Name: 刘嘉: Page :3   Item :7
Name: 孙保镇: Page :3   Item :8
Name: 陈文艳: Page :3   Item :9
Name: 于文龙: Page :3   Item :10
Name: 于文龙: Page :3   Item :11
Name: 朱世飞: Page :3   Item :24
Name: 李倩云: Page :3   Item :29
Name: 闫静: Page :3   Item :30
Name: 李平: Page :3   Item :31
Name: 李阳: Page :4   Item :2
Name: 杨辉: Page :4   Item :7
Name: 王淑霞: Page :4   Item :11
Name: 李学

Name: 薄文杰: Page :26   Item :21
Name: 郭玉发: Page :26   Item :28
Name: 白科硕: Page :26   Item :29
Name: 韩书森: Page :27   Item :4
Name: 苗建光: Page :27   Item :9
Name: 董志旺: Page :27   Item :10
Name: 姚荣水: Page :27   Item :11
Name: 侯磊: Page :27   Item :12
Name: 邢辉: Page :27   Item :16
Name: 付宝力: Page :27   Item :25
Name: 马静: Page :28   Item :2
Name: 崔禧华: Page :28   Item :3
Name: 王印坤: Page :28   Item :4
Name: 田宝江: Page :28   Item :5
Name: 郑兆健: Page :28   Item :6
Name: 刘鸿: Page :28   Item :8
Name: 王观涛: Page :28   Item :9
Name: 王秀娥: Page :28   Item :21
Name: 王桂清: Page :28   Item :22
Name: 张馨予: Page :28   Item :23
Name: 肖永军: Page :28   Item :24
Name: 韩庆栋: Page :29   Item :2
Name: 李亚平: Page :29   Item :3
Name: 王连芹: Page :29   Item :4
Name: 王汉勇: Page :29   Item :5
Name: 王晨: Page :29   Item :9
Name: 李仁: Page :29   Item :16
Name: 栾宝泽: Page :29   Item :22
Name: 汪一斌: Page :29   Item :23
Name: 吴建彤: Page :29   Item :25
Name: 董红燕: Page :29   Item :28
Name: 田金明: Page :29   Item :30
Name: 梁爽: Page :30   Item :2

Name: 丁士海: Page :54   Item :15
Name: 赵得名: Page :54   Item :22
Name: 刘月武: Page :54   Item :27
Name: 潘艳华: Page :54   Item :28
Name: 济南尊旭商务服务合伙企业（有限合伙）: Page :54   Item :31
Name: 裴冬: Page :55   Item :0
Name: 王帆: Page :55   Item :1
Name: 李新慧: Page :55   Item :2
Name: 天津鼎智顺拍企业管理有限公司: Page :55   Item :5
Name: 王晨: Page :55   Item :10
Name: 陈敬华: Page :55   Item :16
Name: 曹式强: Page :55   Item :17
Name: 侯淑芹: Page :55   Item :18
Name: 何文荣: Page :55   Item :24
Name: 任秀哲: Page :55   Item :26
Name: 李丹丹: Page :56   Item :8
Name: 崔金鹏: Page :56   Item :9
Name: 陈则艳: Page :56   Item :10
Name: 董志远: Page :56   Item :11
Name: 李学旺: Page :56   Item :13
Name: 崔庆伟: Page :56   Item :14
Name: 王伟: Page :56   Item :16
Name: 毕桂娟: Page :56   Item :20
Name: 张荣强: Page :56   Item :21
Name: 张勇: Page :56   Item :22
Name: 方建宁: Page :56   Item :30
Name: 腾洪霞: Page :57   Item :0
Name: 刘杰: Page :57   Item :3
Name: 卢涛: Page :57   Item :4
Name: 闫高健: Page :57   Item :6
Name: 刘志平: Page :57   Item :10
Name: 于富豪: Page :57   Item :13

Name: 李滨: Page :76   Item :26
Name: 石晶晶: Page :76   Item :28
Name: 付宝力: Page :76   Item :29
Name: 查升博: Page :76   Item :31
Name: 李金玲: Page :77   Item :4
Name: 尹天才: Page :77   Item :5
Name: 张华义: Page :77   Item :10
Name: 郝连欣: Page :77   Item :15
Name: 张凯: Page :77   Item :16
Name: 贾林: Page :77   Item :18
Name: 王双双: Page :77   Item :29
Name: 周晓军: Page :77   Item :30
Name: 叶超: Page :77   Item :31
Name: 葛凯悦: Page :78   Item :4
Name: 刘进华: Page :78   Item :5
Name: 刘进华: Page :78   Item :6
Name: 王凤彬: Page :78   Item :7
Name: 郑铁成: Page :78   Item :8
Name: 索连洋: Page :78   Item :10
Name: 王桂清: Page :78   Item :11
Name: 刘睿: Page :78   Item :15
Name: 韩秀丽: Page :78   Item :17
Name: 孙维义: Page :78   Item :18
Name: 郑远: Page :78   Item :19
Name: 吴君: Page :78   Item :28
Name: 刘明利: Page :78   Item :31
Name: 靳延海: Page :79   Item :2
Name: 商学惠: Page :79   Item :8
Name: 雷万林: Page :79   Item :9
Name: 周晓玲: Page :79   Item :26
Name: 宁圃春: Page :79   Item :27
Name: 张涛: Page :79   Item :30
Name: 艾学敏: Page :80   Item

In [10]:
driver.quit()

In [11]:
csvFile.close()
csv = pd.read_csv('namelist.csv', encoding='utf-8')
csv.to_excel('namelist.xlsx', sheet_name='data')

In [12]:
t2 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t2))

seconds = t2 - t1
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))


01:08:13


In [13]:
#            print("URL: "+driver.current_url)
#             print("id:"+str(bidInfo["id"])+" title: "+bidInfo["title"])
#             print("consultPrice:"+str(bidInfo["consultPrice"]) + " currentPrice:" +str(bidInfo["currentPrice"]))
#             print("bidCount:"+str(bidInfo["bidCount"]) +" delayCount:"+str(bidInfo["delayCount"])+" applyCount:"+str(bidInfo["applyCount"]))
#             print("date:"+bidInfo["date"])

In [14]:
#^[1-9]\d*\.\d*|0\.\d*[1-9]\d*$


In [15]:
# match = r"\d+\.\d*"

# areas = re.findall(match,"abc12121223ac-1.12dddd")

# areas

